In [1]:
import pandas as pd

data = pd.read_csv('../뉴스 데이터/뉴스 크롤링/라벨링/실험.csv', encoding='cp949')
data.head()

,단어,개수,label
0,열린,109,1
1,최종,51,1
2,우승을,6,1
3,경기가,7,1
4,이상의,46,1


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('../뉴스 데이터/뉴스 크롤링/라벨링/실험.csv', encoding='cp949')
X = data.iloc[:, :]
y = data.iloc[:, :]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [4]:
import re

X_train['단어'] = X_train['단어'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))
X_train.head()

,단어,개수,label
17,목표로,60,1
67,큰,160,1
124,기대했다,13,1
181,상황이다,12,-1
13,진행할,63,1


In [5]:
from konlpy.tag import Okt
okt = Okt()

def okt_tokenizer(text):
     tokens = okt.morphs(text) # 형태소 단위로 구분 분석
     return tokens


from sklearn.feature_extraction.text import TfidfVectorizer #TF-idf 기반 벡터화

tfidf = TfidfVectorizer(tokenizer = okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf.fit(X_train['단어'])
train_tfidf = tfidf.transform(X_train['단어'])

c:\Users\PC\anaconda3\envs\pandas-dev\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [6]:
from sklearn.linear_model import LogisticRegression

SA_lr = LogisticRegression(random_state = 0)
SA_lr.fit(train_tfidf, X_train['label'])

#사이킷런: 로지스틱 회귀 실습

LogisticRegression(random_state=0)

In [7]:
from sklearn.model_selection import GridSearchCV

params = {'C': [1, 3, 3.5, 4, 4.5, 5]}
SA_lr_grid_cv = GridSearchCV(SA_lr, param_grid=params, cv=3, scoring='accuracy', verbose=1)

In [9]:
SA_lr_grid_cv.fit(train_tfidf, X_train['label'])

# 모델 학습시키기


Fitting 3 folds for each of 6 candidates, totalling 18 fits


GridSearchCV(cv=3, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1, 3, 3.5, 4, 4.5, 5]}, scoring='accuracy',
             verbose=1)

In [10]:
print(SA_lr_grid_cv.best_params_, round(SA_lr_grid_cv.best_score_, 4))

{'C': 3} 0.7858


In [11]:
SA_lr_best = SA_lr_grid_cv.best_estimator_

In [13]:
test_tfidf = tfidf.transform(y_test['단어'])

In [14]:
test_predict = SA_lr_best.predict(test_tfidf)

In [15]:
from sklearn.metrics import accuracy_score

print('감정 분류 정확도 : ', round(accuracy_score(y_test['label'], test_predict), 3))

감정 분류 정확도 :  0.63
